In [1]:
%matplotlib notebook

from __future__ import division
import math
from scipy.special import zeta
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

NORM_CONSTANT = zeta(3,1)
def prob(n):
    f = lambda n : 1 / (n*n*n) / NORM_CONSTANT
    f.supp = float('inf')
    return f

def mkprob(ns):
    l = len(ns)
    f = lambda n : 1.0/l if n in ns else 0.0
    f.supp = max(ns)
    return f

def integral(p):
    prev = float('inf')
    acc = 0.0
    n = 1
    while 1 - acc > 1e-7:
        prev, acc, n = acc, acc + p(n), n + 1
    return acc

def expect(p):
    if p.supp == float('inf'):
        crit = lambda acc, prev, n: acc - prev > 1e-7
    else:
        crit = lambda acc, prev, n: n < p.supp
    acc = 0.0
    prev = 1.0
    n = 1
    while crit(acc, prev, n):
        prev, acc, n = acc, acc + n * p(n), n + 1
    return acc

def c_trans(p):
    def pre_p(n):
        # n >= 1
        if n > 1 and (n - 1) % 3 == 0:
            v = p((n - 1) // 3)
            if v > 0:
                return v
        return p(2 * n)
    if p.supp == float('inf'):
        supp = float('inf')
    else:
        supp = 3 * p.supp + 1
    if supp == float('inf'):
        crit = lambda acc, prev, n: acc - prev > 1e-7
    else:
        crit = lambda acc, prev, n: n < supp
    acc = 0.0
    prev = 1.0
    n = 1
    while crit(acc, prev, n):
        prev, acc, n = acc, acc + pre_p(n), n + 1
    f = lambda n: pre_p(n)/acc
    f.supp = supp
    return f

In [2]:
def plot_1():
    p_i = mkprob([7])
    v = []
    for i in range(5):
        v.append((i, expect(p_i)))
        p_i = c_trans(p_i)
    v = np.array(v)
    plt.plot(v[:,0], np.log(v[:,1]))
    plt.show()
plot_1()

ZeroDivisionError: float division by zero

**so it looks like the expectation is growing!!**

In [ ]:
def plot_2():
    p_i = prob
    for i in range(50):
        if i % 5 == 0:
            v = [] 
            for n in range(100):
                v.append((n+1, p_i(n+1)))
            v = np.array(v)
            plt.scatter(v[:10,0], v[:10,1], alpha=0.5, s=2)
        p_i = c_trans(p_i)
    plt.show()
    print v[:10,1]

#plot_2()

**but all needs to be taken w/ a grain of salt: exact numerical calculations are probably different**

In [ ]:
print 1/NORM_CONSTANT

In [ ]:
print 1/zeta(4,1)